In [6]:
import pprint
from collections import defaultdict
import numpy as np
import pandas as pd

class DiscountFactor:
    def __init__(self, gamma, epsilon):
        self._gamma = gamma
        self._epsilon = epsilon
        
    def verifica_convergencia(self, Vk, Vk1):
        return True if np.abs(Vk - Vk1) < self._epsilon ** ((1-self._gamma)/2*self._gamma) else False

class Graph(object):
    """ Graph data structure, undirected by default. """

    def __init__(self, connections, gamma, epsilon, log=False):
        self._graph = defaultdict(set)
        self._graph_rewards = defaultdict(dict)
        self._graph_probs = defaultdict(dict)
        
        self._gamma = gamma
        self._epsilon = epsilon
        self._disc_factor = DiscountFactor(self._gamma, self._epsilon)
        
        self._log = log
        self.add_connections(connections)

    def add_connections(self, connections):
        """ Add connections (list of tuple pairs) to graph """

        for node1, node2, reward, prob, directed in connections:
            self.add(node1, node2, reward, prob, directed)

    def add(self, node1, node2, reward, prob, directed=False):
        """ Add connection between node1 and node2 """

        self._graph[node1].add(node2)
        self._graph_rewards[node1][node2] = reward
        self._graph_probs[node1][node2] = prob
        
        if not directed:
            self._graph[node2].add(node1)
            self._graph_rewards[node2][node1] = reward
            self._graph_probs[node2][node1] = prob

    def remove(self, node):
        """ Remove all references to node """

        for n, cxns in self._graph.items():  # python3: items(); python2: iteritems()
            try:
                cxns.remove(node)
            except KeyError:
                pass
        try:
            del self._graph[node]
        except KeyError:
            pass

    def is_connected(self, node1, node2):
        """ Is node1 directly connected to node2 """

        return node1 in self._graph and node2 in self._graph[node1]

    def find_path(self, node1, node2, path=[]):
        """ Find any path between node1 and node2 (may not be shortest) """

        path = path + [node1]
        if node1 == node2:
            return path
        if node1 not in self._graph:
            return None
        for node in self._graph[node1]:
            if node not in path:
                new_path = self.find_path(node, node2, path)
                if new_path:
                    return new_path
        return None

    def find_path_by_time(self, time):
        res = pd.DataFrame(index=g._graph.keys())
        res_df = pd.DataFrame(index=g._graph.keys())
        conv = pd.DataFrame(index=g._graph.keys())
        
        for node in g._graph.keys():
            total_rewards = []
            
            for t in range(1, time):
                reward, reward_df, path = self.get_max_reward(node, 1, t, [], [], [], node)
                if self._log: print(f'Node Atual: {node} / Time: {t}' + \
                                    f'/ Max Reward: {np.max(reward)} / Path: {path[np.argmax(reward_df)]}')
                
                res.loc[node, t] = np.max(reward)
                res_df.loc[node, t] = np.max(reward_df)
                
                if t > 1:
                    bool_conv = self._disc_factor.verifica_convergencia(res_df.loc[node, t-1], res_df.loc[node, t])
                    conv.loc[node, t] = bool_conv
            
            if self._log: print('-> Changing Node ---')
            
        return res, res_df, conv
    
    def get_max_reward(self, node, time_inicial, time_final, reward=[], reward_df=[], path=[], pai=None, reward_atual=0):
        if time_inicial > time_final:
            return reward, reward_df, path
        
        for vizinho in self._graph[node]:
            r = self._graph_rewards[node][vizinho] + reward_atual
            r_df = self._graph_rewards[node][vizinho] * self._gamma**time_inicial + reward_atual
            
            if time_inicial == time_final:
                reward += [r]
                reward_df += [r * self._gamma**time_inicial]
                path += [f'{pai} -> {vizinho}']
            
            reward, reward_df, path = self.get_max_reward(vizinho, time_inicial+1, time_final, 
                                                          reward, reward_df, path, f'{pai} -> {vizinho}', r)
            
        return reward, reward_df, path
    
    def __str__(self):
        return '{}({})'.format(self.__class__.__name__, dict(self._graph))

In [7]:
_epsilon = 0.001
_gamma = 0.9
print('Threshold:', _epsilon ** ((1-_gamma)/2*_gamma))

bool_log = False
conn = [('A', 'B', -1, 1, False), 
        ('A', 'C', -5, 1, False), 
        ('B', 'C', -2, 1, False), 
        ('B', 'D', -1, 1, False), 
        ('C', 'D', -5, 1, False), 
        ('C', 'E', -1, 1, False), 
        ('C', 'F', -3, 1, False), 
        ('D', 'F', -5, 1, False), 
        ('E', 'F', -1, 1, False),
        ('D', 'A', 3, 0.6, True),
        ('D', 'E', 3, 0.4, True)]

g = Graph(conn, _gamma, _epsilon)
print(g)
print('---')

res, res_df, conv = g.find_path_by_time(10)

Threshold: 0.7328245331389042
Graph({'A': {'B', 'C'}, 'B': {'A', 'C', 'D'}, 'C': {'F', 'D', 'B', 'A', 'E'}, 'D': {'F', 'C', 'B', 'A', 'E'}, 'E': {'F', 'C'}, 'F': {'E', 'C', 'D'}})
---


In [8]:
res

,1,2,3,4,5,6,7,8,9
A,-1.0,-2.0,1.0,0.0,-1.0,2.0,1.0,0.0,3.0
B,-1.0,2.0,1.0,0.0,3.0,2.0,1.0,4.0,3.0
C,-1.0,-2.0,0.0,-1.0,-1.0,1.0,0.0,0.0,2.0
D,3.0,2.0,1.0,4.0,3.0,2.0,5.0,4.0,3.0
E,-1.0,-2.0,-3.0,-1.0,-2.0,-2.0,0.0,-1.0,-1.0
F,-1.0,-2.0,-3.0,-3.0,-1.0,-2.0,-2.0,0.0,-1.0


In [9]:
res_df

,1,2,3,4,5,6,7,8,9
A,-0.9,-1.62,0.729,0.0000,-0.59049,1.062882,0.478297,0.000000,1.162261
B,-0.9,1.62,0.729,0.0000,1.77147,1.062882,0.478297,1.721869,1.162261
C,-0.9,-1.62,0.000,-0.6561,-0.59049,0.531441,0.000000,0.000000,0.774841
D,2.7,1.62,0.729,2.6244,1.77147,1.062882,2.391485,1.721869,1.162261
E,-0.9,-1.62,-2.187,-0.6561,-1.18098,-1.062882,0.000000,-0.430467,-0.387420
F,-0.9,-1.62,-2.187,-1.9683,-0.59049,-1.062882,-0.956594,0.000000,-0.387420


In [10]:
conv

,2,3,4,5,6,7,8,9
A,True,False,True,True,False,True,True,False
B,False,False,True,False,True,True,False,True
C,True,False,True,True,False,True,True,False
D,False,False,False,False,True,False,True,True
E,True,True,False,True,True,False,True,True
F,True,True,True,False,True,True,False,True
